<a href="https://colab.research.google.com/github/vivekmithlesh/GSOC_MIC_TEST/blob/main/Social_Media_Data_Extraction_%26_Preprocessing_(API_Handling_%26_Text_Cleaning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tweepy emoji nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.8 MB/s eta 0:00:00


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import tweepy

BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAFudwgEAAAAA%2FI4rMWOC3rNGnJ2dK9wm6VFsuGA%3DBGNRaAl9R7nw48e8iHyTDatpd4NXXBHtNqiezBbKQWhrVzywwJ'

client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [4]:
keywords = [
    "suicidal", "depressed", "anxiety attack", "panic attack",
    "mental breakdown", "overwhelmed", "addiction help",
    "relapse", "hopeless", "i want to die", "i hate my life",
    "i feel lost", "can’t cope", "end my life", "kill myself"
]

query = ' OR '.join(keywords) + ' lang:en -is:retweet'

In [5]:
tweets = client.search_recent_tweets(
    query=query,
    tweet_fields=['created_at', 'public_metrics'],
    max_results=100
)

In [6]:
import re
import emoji
import pandas as pd
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = emoji.replace_emoji(text, replace='')  # Remove emojis
    text = re.sub(r"[^A-Za-z0-9\s]", "", text.lower())  # Remove special characters
    return ' '.join([word for word in text.split() if word not in stop_words])

In [7]:
data = []
for tweet in tweets.data:
    metrics = tweet.public_metrics
    data.append({
        'post_id': tweet.id,
        'timestamp': tweet.created_at,
        'content': tweet.text,
        'cleaned_content': clean_text(tweet.text),
        'likes': metrics['like_count'],
        'retweets': metrics['retweet_count']
    })

df = pd.DataFrame(data)
df.head()

,post_id,timestamp,content,cleaned_content,likes,retweets
0,1909293034514063580,2025-04-07 17:11:47+00:00,Portnoy is hopeless. He would rather lose 10 t...,portnoy hopeless would rather lose 10 15 portf...,0,0
1,1909293033876599291,2025-04-07 17:11:46+00:00,It’s hard to care about all this shit cus it’s...,hard care shit cus damn near monthly something...,0,0
2,1909293033004450050,2025-04-07 17:11:46+00:00,Suddenly I’m not depressed anymore https://t.c...,suddenly im depressed anymore,0,0
3,1909293029967478936,2025-04-07 17:11:45+00:00,"According to common sense, the theory goes fro...",according common sense theory goes verificatio...,0,0
4,1909293028759527575,2025-04-07 17:11:45+00:00,RT @Bubblebathgirl: Unhinged leftist protester...,rt bubblebathgirl unhinged leftist protester s...,0,155


In [8]:
df.to_csv("task1_cleaned_data.csv", index=False)

from google.colab import files
files.download("task1_cleaned_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>